In [1]:
from random import shuffle
import multiprocessing
from multiprocessing import Pool
import csv
import re
import numpy as np
import pandas as pd

In [3]:
import sagemaker
from sagemaker import get_execution_role
import json
import boto3

sess = sagemaker.Session()

role = get_execution_role()
print(role) # This is the role that SageMaker would use to leverage AWS resources (S3, CloudWatch) on your behalf

bucket = sess.default_bucket() # Replace with your own bucket name if needed

prefix = 'review_topics' #Replace with the prefix under which you want to store the data if needed

arn:aws:iam::443501626368:role/service-role/AmazonSageMaker-ExecutionRole-20200806T142735


Final function

In [ ]:
# define a simple tokenizer (NLTK won't be available to us later on, in our Lambda function)

def simple_tokenizer(input_text):
    REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(=)|(`)")
    REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)|(\n)|(\t)")
    tokens = REPLACE_NO_SPACE.sub("", input_text.lower())
    tokens = REPLACE_WITH_SPACE.sub(" ", tokens) # note that blazing text expects space-separated tokens
    return tokens

In [42]:
# read and process the dataset
def preprocesser(input_file, output_file):
    filepath=Path.joinpath(Path.cwd().parent, 'data', 'amazon_review_polarity_csv', input_file)
    df = pd.read_csv(filepath, names=["Label", "Title", "Review"])
    # class 0 is the negative and class 1 is the positive
    df['Label']=train_df['Label']-1
    # drop Title, focus on review itself
    df=df.drop('Title', axis=1)
    # Shuffle reviews and corresponding labels within training and test sets
    df = df.sample(frac = 1)
    # apply the simple tokenizer
    df['Review']=df['Review'].apply(simple_tokenizer)
    # Prefix the index-ed label with __label__
    df['Label']=df['Label'].apply(lambda row: "__label__" +  str(row) )
    # convert the transformed dataframe into a list
    transformed_rows = np.array(df).tolist()
    # write to csv file
    with open(output_file, 'w') as csvoutfile:
        csv_writer = csv.writer(output_file, delimiter=' ', lineterminator='\n') # notice the delimiter.
        csv_writer.writerows(transformed_rows)

In [ ]:
%%time

# Preparing the training dataset
preprocess('train.csv', 'polarity.train')

In [ ]:
%%time

# Preparing the validation dataset        
preprocess('test.csv', 'polarity.validation')

#### Preparing and Processing the training data

In [41]:
from pathlib import Path
filename='train.csv'
print(Path.cwd().parent)
print(Path.joinpath(Path.cwd().parent, 'data', 'amazon_review_polarity_csv', filename))

/home/ec2-user/SageMaker
/home/ec2-user/SageMaker/data/amazon_review_polarity_csv/train.csv


In [5]:
# class 0 is the negative and class 1 is the positive
train_df['Label']=train_df['Label']-1
train_df['Label'].value_counts()

1    1800000
0    1800000
Name: Label, dtype: int64

In [6]:
# drop Title, focus on review itself
train_df=train_df.drop('Title', axis=1)
train_df.head()

,Label,Review
0,1,This sound track was beautiful! It paints the ...
1,1,I'm reading a lot of reviews saying that this ...
2,1,This soundtrack is my favorite music of all ti...
3,1,I truly like this soundtrack and I enjoy video...
4,1,"If you've played the game, you know how divine..."


In [32]:
#Shuffle reviews and corresponding labels within training and test sets
train_df = train_df.sample(frac = 1)

,Label,Review
1136550,0,I have had the old style littermaids for years...
3399067,0,I assumed this dance pad would be decent due t...
2061393,1,This is a great book if you want to pass the S...
544960,0,I purchased this air pump seeking a pump that ...
3199043,0,"Right out of the box, this product felt flimsy..."
...,...,...
2361291,0,"This coffee was very stale, it had no expirati..."
3437035,1,I purchased this for a friend and i was very p...
161670,1,I believe that the Dream Dance collection is c...
574524,0,What a bore! If you really want a magical read...


In [24]:
# define a simple tokenizer (NLTK won't be available to us later on, in our Lambda function)

def simple_tokenizer(input_text):
    REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(=)|(`)")
    REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)|(\n)|(\t)")
    tokens = REPLACE_NO_SPACE.sub("", input_text.lower())
    tokens = REPLACE_WITH_SPACE.sub(" ", tokens) # note that blazing text expects space-separated tokens
    return tokens

In [25]:
# apply the simple tokenizer
train['Review']=train['Review'].apply(simple_tokenizer)

In [ ]:
# Prefix the index-ed label with __label__
train['Label']=train['Label'].apply(lambda row: "__label__" +  str(row) )

In [26]:
train.head()

,Label,Review
871811,__label__0,i cant believe the number of people tripping a...
1147345,__label__1,i bought this book to make ice cream with a ki...
1527049,__label__0,worse u2 album ive ever heard there is nothing...
1253689,__label__0,i have no idea what this man is talking about ...
2085461,__label__1,i live much of the year in the great basins an...


In [33]:
# convert the transformed dataframe into a list
transformed_rows = np.array(train_df).tolist()

In [36]:
# blazing text requires space-separated word tokens.
output_file = 'polarity.train'
with open(output_file, 'w') as csvoutfile:
    csv_writer = csv.writer(csvoutfile, delimiter=' ', lineterminator='\n') # notice the delimiter.
    csv_writer.writerows(transformed_rows)